In [34]:
from openai import OpenAI
import json 
import pandas as pd
import time
import credentials 
from youtube_transcript_api import YouTubeTranscriptApi
import os

imported_key = credentials.OPENAI_KEY
openai.api_key = imported_key
# model_engine="text-davinci-003"

# max_summaries = 10
data_folder = "data"
#Replace with "data folder" when doing final run - will incur OpenAI charges for token usage
# max_summaries = 250
# data_folder = "data"
# in_filepath = f"{data_folder}/01_youtubeStatsTranscripts.json"

# summ_filepath = f"{data_folder}/summaries"
# if not os.path.exists(summ_filepath):
#     os.makedirs(summ_filepath)
    
# qa_filepath = f"{data_folder}/QA_out"
# if not os.path.exists(qa_filepath):
#     os.makedirs(qa_filepath)
    



client = OpenAI(
    # This is the default and can be omitted
    api_key=imported_key
)

In [35]:
input_filename = f"{data_folder}/01_youtube_stats_df.csv"
input_df = pd.read_csv(input_filename)
input_df.head()

,Unnamed: 0,video_id,published_at,channel_id,channel_name,Category,favourites,title,views,likes,dislikes,comment_count,description,tags
0,0,i60mJkO63wE,2025-03-11T21:25:49Z,UC5UjcZoCX9sr6ox_-7bXGzw,Yohana Ortiz,People & Blogs,0,I QUIT MY JOB...,48453,3526,Not available,248,"HELLO AMIGAS, I hope I've answered any questio...",No Tags
1,1,GlgRsFpC0n8,2025-01-14T23:30:07Z,UCDq5VuCYsWfjihGdY4i09hg,Nicole Unscripted,People & Blogs,0,"I Quit My Job, Burnout, Menopause, Grief | Lif...",51863,2094,Not available,647,Sharing part of my personal journey as a singl...,No Tags
2,2,bEH3sL4unP8,2025-01-24T16:42:28Z,UC30XXKvIYzDhqWjM5nhorEA,Pauly Walnuts,Gaming,0,I just quit my job,313604,16485,Not available,3229,"In this video, I shared my story of quitting m...",No Tags
3,3,UxC9AyKQ65w,2025-01-14T17:00:30Z,UC9MOzHsx_mfEeS7OYh-6ykA,Varun Rana,People & Blogs,0,I Quit My Job Without a Backup Plan,13491,855,Not available,169,Telling the story of the time I quit my high-p...,No Tags
4,4,NErXzkS_qBc,2024-01-29T16:00:41Z,UCLCW9rn6lwZOHDnPnrWGTDA,Im.mattmims,People & Blogs,0,Today I Quit My Job... A Rant On Mental Heath ...,33333,1549,Not available,637,Today I quit my job. In this raw and honest vi...,"['Matt Mims', 'Youtube 2024', 'Matthew Mims', ..."


In [36]:
# with open(in_filepath, 'r') as f:
#     data = json.load(f)

#### WORKING - MOVE TO 02?
##Get transcripts using get_transcript() of YoutubeTranscriptApi for each transcript
### Get as dictionary of dictionary: youtube_id : {}

transcript_dict = {}
yt_api = YouTubeTranscriptApi()

for i, video_id in enumerate(input_df['video_id']):
    transcript_sents = None #create new list for individual sentences
    transcript_joined = None
    try:
        
        curr_result = yt_api.get_transcript(video_id,languages=['en'])
        # print(curr_result)
        transcript_sents = [str(x['text']).replace("\xa0", "") for x in curr_result]
        transcript_joined = " ".join(transcript_sents)
        print(f"WORKING - {video_id}")
        print(f"FIRST SENTS FOR {video_id} TRANSCRIPT: {transcript_sents[:5]}")
        
        
    except:
        print(f"MISSING {video_id} - SKIPPED")
    transcript_dict[video_id] = {
                                    "joined": transcript_joined,
                                    "sents": transcript_sents
        
                                }

    if i == 5:
        break
    # if i % 50 == 0:
    #     print(f"PROCESSED {i} TRANSCRIPTS")
    #     print(f"LENGTH OF DICTS")

# print(len(transcript_dict))
    



WORKING - i60mJkO63wE
FIRST SENTS FOR i60mJkO63wE TRANSCRIPT: ['hello Amigas welcome back to my Channel', 'today I have a different type of video', "than what I normally film um we're going", 'to be having a little sit down talk uh', 'about something really serious you guys']
WORKING - GlgRsFpC0n8
FIRST SENTS FOR GlgRsFpC0n8 TRANSCRIPT: ["yeah y'all I quit my", '[Music]', 'job yall think about', 'how about a year and a half two years', 'ago when I went on my little journey and']
WORKING - bEH3sL4unP8
FIRST SENTS FOR bEH3sL4unP8 TRANSCRIPT: ['all right guys so I just quit my', "corporate job I've been there for three", 'years and I have zero backup plan um so', 'I for some background worked for a tech', 'company in New York City and I honestly']
WORKING - UxC9AyKQ65w
FIRST SENTS FOR UxC9AyKQ65w TRANSCRIPT: ['I got burnt out at my first job out of', 'college and not only did it suck the', 'life out of me but it made me question', 'my entire existence so after two', 'draining years instea

In [37]:
for curr_key in transcript_dict.keys():
    try:
        print(f"youtube_id: {curr_key}, number of sents: {len(transcript_dict[curr_key]['sents'])}")
        print(f"first sents{transcript_dict[curr_key]['sents'][:5]}")
        print(f"number of tokens: {len(transcript_dict[curr_key]['joined'].split())}")
        print("----")
    except:
        print(f"TRANSCRIPT MISSING FOR {curr_key}") 

print(transcript_dict)

youtube_id: i60mJkO63wE, number of sents: 257
first sents['hello Amigas welcome back to my Channel', 'today I have a different type of video', "than what I normally film um we're going", 'to be having a little sit down talk uh', 'about something really serious you guys']
number of tokens: 1912
----
youtube_id: GlgRsFpC0n8, number of sents: 231
first sents["yeah y'all I quit my", '[Music]', 'job yall think about', 'how about a year and a half two years', 'ago when I went on my little journey and']
number of tokens: 1746
----
youtube_id: bEH3sL4unP8, number of sents: 286
first sents['all right guys so I just quit my', "corporate job I've been there for three", 'years and I have zero backup plan um so', 'I for some background worked for a tech', 'company in New York City and I honestly']
number of tokens: 2201
----
youtube_id: UxC9AyKQ65w, number of sents: 591
first sents['I got burnt out at my first job out of', 'college and not only did it suck the', 'life out of me but it made me quest

In [ ]:
### TO DO: verify transcript dict and save to data


In [40]:
transcripts_json_outname = f"{data_folder}/02a_transcripts.json"
with open(transcripts_json_outname, 'w', encoding='utf-8') as f:
    json.dump(transcript_dict, f, ensure_ascii=False, indent=4)

In [24]:
#### NEW EXAMPLE - need to fix the limits and billing on OPENAI website: https://platform.openai.com/settings/organization/billing/overview 
#### Source: https://cookbook.openai.com/examples/batch_processing
categorize_system_prompt = """
                You are a text analysis assistant, skilled in identifying key arguments and summarizing text transcripts with brevity and neutrality.
                
                The following transcript is from a YoutTbe video concerning their decision to quit their job. 
                Please identify the following items from the text: current job, former job, 1-3 primary reasons for them quitting their job, and 1-3 emotions that the speaker is feeling. 
                Please output a json object containing the following information:

                {
                    current_job: string // the current job of the speaker
                    previous_job: string // the job that the speaker most recently quit
                    quit_reason_main: string // 1-sentence summary of the primary reason that the speaker quit their job
                    emotions_list: string [] // list of emotions that the speaker has displayed 
                    emotions_dict: dictionary {emotion_name: cause_of_emotion} // dictionary where the key is an emotion that the speaker exhibits, and the value is a 1-sentence summary of why the speaker feels this emotion. 
                }

                quit_reason_main should include the main reason that the speaker provided for why they quit their job - be specific as possible.
                emotions_list can include multiple emotions, but try to keep it to under 3-5 and keep them simple and only use lowercase letters.
                emotions_dict can include multiple emotions, but try to keep it to under 3-5. For each key-value pair in emotions_dict, the key should be simple and only use lower case letters. 
                
                """

def get_categories(description):
    response = client.chat.completions.create(
    model="gpt-4o-mini",
    temperature=0.1,
    # This is to enable JSON mode, making sure responses are valid json objects
    response_format={ 
        "type": "json_object"
    },
    messages=[
        {
            "role": "system",
            "content": categorize_system_prompt
        },
        {
            "role": "user",
            "content": description
        }
    ],
    )

    return response.choices[0].message.content


In [42]:
### Create qa_dict 
qa_dict = {}
for video_id in transcript_dict.keys():
    curr_transcript = transcript_dict[video_id]['joined']
    result = get_categories(curr_transcript)
    qa_dict[video_id] = result
    print(result)
    print("---")

{
    "current_job": "social media influencer",
    "previous_job": "teacher",
    "quit_reason_main": "The speaker quit their job due to ongoing discomfort and fear stemming from negative attention related to their social media presence, which made them feel like they were walking on eggshells at work.",
    "emotions_list": ["fear", "relief", "sadness"],
    "emotions_dict": {
        "fear": "The speaker felt fear due to the possibility of receiving more negative emails about their social media activities.",
        "relief": "The speaker experienced relief after making the decision to quit, feeling a tension release from their body.",
        "sadness": "The speaker felt sadness about leaving a job they loved and the emotional toll of the situation."
    }
}
---
{
    "current_job": "medical transcriptionist",
    "previous_job": "medical transcriptionist",
    "quit_reason_main": "The speaker quit their job due to severe burnout that was negatively impacting their health.",
    "e

In [52]:
qa_dict

{'i60mJkO63wE': '{\n    "current_job": "social media influencer",\n    "previous_job": "teacher",\n    "quit_reason_main": "The speaker quit their job due to ongoing discomfort and fear stemming from negative attention related to their social media presence, which made them feel like they were walking on eggshells at work.",\n    "emotions_list": ["fear", "relief", "sadness"],\n    "emotions_dict": {\n        "fear": "The speaker felt fear due to the possibility of receiving more negative emails about their social media activities.",\n        "relief": "The speaker experienced relief after making the decision to quit, feeling a tension release from their body.",\n        "sadness": "The speaker felt sadness about leaving a job they loved and the emotional toll of the situation."\n    }\n}',
 'GlgRsFpC0n8': '{\n    "current_job": "medical transcriptionist",\n    "previous_job": "medical transcriptionist",\n    "quit_reason_main": "The speaker quit their job due to severe burnout that wa

In [56]:
#### WORKING - Make qa_dict into flat JSOn to create the dataframe

qa_dict_final = {
    "video_id": [],
    "current_job": [],
    "previous_job": [],
    "quit_reason_main":[],
    "emotions_list":[],
    "emotions_dict":[],

}

for video_id, results in qa_dict.items():
    qa_dict_final['video_id'].append(video_id)
    # print(results)
    results_json = json.loads(results)
    for curr_col, curr_val in results_json.items():
        qa_dict_final[curr_col].append(curr_val)
        
    

,video_id,current_job,previous_job,quit_reason_main,emotions_list,emotions_dict
0,i60mJkO63wE,social media influencer,teacher,The speaker quit their job due to ongoing disc...,"[fear, relief, sadness]",{'fear': 'The speaker felt fear due to the pos...
1,GlgRsFpC0n8,medical transcriptionist,medical transcriptionist,The speaker quit their job due to severe burno...,"[relief, freedom, stress]",{'relief': 'The speaker felt relief after quit...
2,bEH3sL4unP8,YouTuber,data analyst at a tech company in New York City,The speaker quit their job due to being unfair...,"[frustration, relief, determination]",{'frustration': 'The speaker felt frustrated d...
3,UxC9AyKQ65w,content creator,software engineer at Amazon,The speaker quit their job due to burnout from...,"[relief, fear, excitement]",{'relief': 'The speaker felt relieved after qu...
4,NErXzkS_qBc,entrepreneur,grocery store employee,The speaker quit their job due to feeling that...,"[frustration, disappointment, determination]",{'frustration': 'The speaker feels frustrated ...
5,AtJEAtZYqVM,unemployed,warehouse distribution center job,The speaker quit due to a toxic work environme...,"[bitter, frustrated, disillusioned]",{'bitter': 'The speaker feels bitter due to th...


In [5]:
# ### OBS - GPT4o mini model can now handle over 10k tokens, no longer requiring chunking

# ####SUMMARIZE LONGER TRANSCRIPTS - store in summaries_dict
# #### save checkpoints to summaries folder
# all_summaries = []
# summaries_dict = {
    
# }
# max_len = 1800
# threshold_perc = 0.75
# threshold = max_len*threshold_perc
# chunksize = 1000
# ind = 0 #used to iterate through transcript chunks to summarize

# # summary_prefix = f"""This is a transcript from a YouTube video.in {threshold} words or less, 
# #                     summarize the following text: """
# # classify_prefix = "This is a transcript from a YouTube video."
# for i, text in enumerate(transcript_dict['sents']):
# # for i,text in enumerate(data['transcript_strings']):
#     video_id = data['videoId'][i]
    
# #     if text and i <=250:
#     if text and i <= max_summaries:
#         tokens = text.split()
# #         print(f"FINAL LENGTH: {len(tokens)}")
#         if len(tokens) >= threshold_perc:
#             summary_chunks = []
#             for i in range(0, len(tokens),chunksize):
#                 ###SET indices to get tokens for each chunk
#                 if i == 0:
# #                     print("---START")
#                     start_ind = i
#                     end_ind = chunksize
#                 elif len(tokens) % i < chunksize and len(tokens)//i <= 1.0:
# #                     print("---END")
#                     start_ind = i +1
#                     end_ind = len(tokens)
#                 else:
# #                     print("---MID")
#                     start_ind = i +1
#                     end_ind = i + chunksize
# #                 print(f"START AND END: {start_ind} to {end_ind}")
# #                 print("****")
#                 to_summarize = " ".join(tokens[start_ind:end_ind])
# #                 print(to_summarize)
# #                 print("****")

#                 summ_query = f"The following is a portion of a YouTube video transcript. Summarize what the speaker is saying and include as many details as you can about the speaker's current and former job positions. Also include details related to their reasons for quitting: {to_summarize}"
# #                 print(len(summ_query.split()))
#                 summary_completion = openai.Completion.create(
#                     engine=model_engine,
#                     prompt=summ_query,
#                     max_tokens=500, #prompt restricts length of response, but max_tokens determines cutoff tokens 
#                     n=1,
#                     stop=None,
#                     temperature=0.5,
#                 )
# #                 print(summary_completion)
#                 summary_chunks.append(summary_completion.choices[0].text)
#             all_summaries.append("\n".join(summary_chunks))
#             summaries_dict[video_id] = "\n".join(summary_chunks)
# #             print(len("\n".join(summary_chunks).split()))
#         else:
#             print("CLASSIFY")
#             continue
#         print("---")
#     else:
# #         print("NO TRANSCRIPT")
#         summaries_dict[video_id] = "NO TRANSCRIPT AVAILABLE"
    
#     ###ADD CHECKPOINT IN 'summaries' file - put last video id into file name
#     if len(str(ind)) ==1:
#         checkpoint_ind = "00" + str(ind)
#     elif len(str(ind)) == 2:
#         checkpoint_ind = "0" + str(ind)
#     else:
#         checkpoint_ind = str(ind)
#     checkpoint_file = f"{summ_filepath}/checkpoint_{checkpoint_ind}_{video_id}.json"
#     with open(checkpoint_file, 'w', encoding='utf-8') as f:
#         json.dump(summaries_dict, f, ensure_ascii=False, indent=4)
    
#     if ind == max_summaries:
#         break
#     ind += 1

---
---
---
---
---
---
---
---
---


In [77]:
# text = summaries.values()
# qa_query = f"This is a transcript from a YouTube video. in three separate bullet points, answer 3 questions for the following transcript: \n What is the current job/occupation of the speaker? \n 2. What was the former job/occupation of the speaker? \n 3. What is the main reason that the speaker had for quitting their job/occupation? \n {text}"
    
# qa_completion = openai.Completion.create(
#     engine=model_engine,
#     prompt=qa_query,
#     max_tokens=500, #prompt restricts length of response, but max_tokens determines cutoff tokens 
#     n=1,
#     stop=None,
#     temperature=0.5,
# )

# qa_result = qa_completion.choices[0].text
# qa_result

In [8]:
# ### OBSOLETE - no longer needed due to GPT4omini version

# #answer questions
# #get last item from chkpt_filpath - last checkpoint w most summaries 
# final_summaries = sorted(os.listdir(summ_filepath))[-1]
# summaries_filepath=f"{summ_filepath}/{final_summaries}"
# with open(summaries_filepath, 'r') as f:
#     summaries = json.load(f)

# qa_dict = {}
# qa_ind = 0 
# for video_id, text in summaries.items():
#     qa_query = f"This is a transcript from a YouTube video. in three separate bullet points, answer 3 questions for the following transcript: \n What is the current job/occupation of the speaker? \n 2. What was the former job/occupation of the speaker? \n 3. What is the main reason that the speaker had for quitting their job/occupation? \n {text}"
#     if len(qa_query.split()) <= 3000:
#         qa_completion = openai.Completion.create(
#             engine=model_engine,
#             prompt=qa_query,
#             max_tokens=500, #prompt restricts length of response, but max_tokens determines cutoff tokens 
#             n=1,
#             stop=None,
#             temperature=0.5,
#         )

#         qa_result = qa_completion.choices[0].text
# #         print(qa_result)
#         if len(qa_result) > 0:
#             qa_dict[video_id] = qa_result
#         else:
#             qa_dict[video_id] = "No Answer Available"

#         if qa_ind % 10 == 0 or qa_ind == 500:
#             if len(str(qa_ind)) ==1:
#                 checkpoint_ind = "00" + str(qa_ind)
#             elif len(str(qa_ind)) == 2:
#                 checkpoint_ind = "0" + str(qa_ind)
#             else:
#                 checkpoint_ind = str(qa_ind)
#             checkpoint_file = f"{qa_filepath}/checkpoint_{checkpoint_ind}_{video_id}.json"

#             with open(checkpoint_file, 'w', encoding='utf-8') as f:
#                 json.dump(qa_dict, f, ensure_ascii=False, indent=4)
#         print("---")
#     else:
#         print("****")
#         print(f'Transcript is too long: {video_id} {len(qa_query.split())}')
#         print(qa_query)
#         continue
#     qa_ind += 1

---
---
---
---
---
---
---
---
---
---
---


In [28]:
# SAVE OUT JSON AND CSV
out_filepath_json = f"{data_folder}/02_QA_results.json"
with open(out_filepath_json, 'w', encoding='utf-8') as f:
    json.dump(qa_dict, f, ensure_ascii=False, indent=4)

In [61]:
qa_dict_final
qa_df = pd.DataFrame(qa_dict_final)
out_filepath_csv = f"{data_folder}/02b_QA_results.csv"
qa_df.to_csv(out_filepath_csv)